In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class LevenChatBot:

    # Dataset 파일을 로드하고 질문과 답변 리스트를 생성
    def __init__(self, filepath):
        self.questions, self.answers = self.load_data(filepath)

    # ChatbotData.csv를 호출하는 함수
    def load_data(self, filepath):
        data = pd.read_csv(filepath)
        questions = data['Q'].tolist()
        answers = data['A'].tolist()
        return questions, answers

    # 두 문장의 레벤슈타인 거리를 계산
    def levenshtein_distance(self, s1, s2):

        # 재귀함수로 비교적 짧은 문장을 행(row)으로 설정하여 시간복잡도를 줄일 수 있음
        if len(s1) < len(s2):
            return self.levenshtein_distance(s2, s1)

        # s2가 더 짧은 문장인 경우 선행 row를 생성
        pre_row = list(range(len(s2)+1))
        for i, c1 in enumerate(s1):
            cur_row = [i + 1]
            for j, c2 in enumerate(s2):
                insert = pre_row[j+1] + 1 # 삽입은 왼쪽 +1
                delete = cur_row[j] + 1 # 삭제는 위쪽 +1
                subs = pre_row[j] + (c1 != c2) # 치환은 대각선에서 c1 != c2 다르면 True(+1) 같으면 False(+0)
                cur_row.append(min(insert, delete, subs)) # 셋 중 최소값을 현재 row에 채워넣음
            pre_row = cur_row
            # 열 순회가 끝나면 현재 row를 선행 row로 바꾸고 다음 row를 작성

        return pre_row[-1] # 최종 row의 마지막 원소를 반환

    # input_sentence에 대한 레벤슈타인 거리를 계산
    def best(self, input_sentence):

        # levenshtein_distance에서 계산된 레벤슈타인 거리값을 저장하는 리스트를 초기화
        similarities = []

        for question in self.questions:
            similarities.append(self.levenshtein_distance(input_sentence, question)) # 질문 리스트를 순회하며 레벤슈타인 거리를 모두 계산해 similarities에 저장

        # similarities를 array로 변환하여 최소값의 인덱스를 호출
        best_index = np.array(similarities).argmin()

        # 레벤슈타인 거리 최소값의 인덱스를 갖는 answers 리스트의 원소를 호출해 반환
        return self.answers[best_index]

In [15]:
# 데이터 파일 경로 지정
filepath = 'ChatbotData.csv'

# 챗봇 객체 생성
chatbot = LevenChatBot(filepath)

# '종료'라고 입력되기 전까지 사용자 입력에 따라 챗봇의 응답을 출력하는 무한 while문
while True:
    input_sentence = input('You: ')
    if input_sentence == '종료':
        break
    response = chatbot.best(input_sentence)
    print('Chatbot: ', response)

You: 숙제 다 했니?
Chatbot:  미리미리 해야죠.
You: 목이 마르다
Chatbot:  물 마시세요.
You: 수고하셨습니다.
Chatbot:  저도 반가워요.
You: 종료
